In [ ]:
from matplotlib import pylab as plt
import datasets
import pickle
import os
from typography_generation.io.crello_util import CrelloProcessor
from typography_generation.tools.denormalizer import Denormalizer
from typography_generation.tools.tokenizer import Tokenizer

from typography_generation.visualization.renderer import TextRenderer

data_dir = "../data"
dataset = datasets.load_from_disk(
    os.path.join(data_dir, "crello_map_features")
)["test"]
# dataset = datasets.load_dataset("cyberagent/crello", revision="3.1")["test"]
fontlabel2fontname = dataset.features["font"].feature.int2str

renderer = TextRenderer(data_dir, fontlabel2fontname)

tokenizer = Tokenizer(data_dir)
dataset_processor = CrelloProcessor(data_dir,tokenizer,dataset,None)
denormalizer = Denormalizer(dataset_processor)


In [ ]:
def load_pred(output_dir):
    fn =f"{output_dir}/logs/prediction.pkl"
    predictions = pickle.load(open(fn,"rb"))
    return predictions
output_dir = "../outs" # please set your output directory here
predictions = load_pred(output_dir)

In [ ]:
from typography_generation.visualization.visualizer import visualize_prediction, visualize_data
import PIL
import numpy as np
import random

# This code shows 3 generated designs per input context

num_sample = 10 

for i in range(num_sample):
    i=random.randint(0,len(dataset))
    N=3
    print(i)
    element_data = dataset[i]
    svg_id = element_data["id"]
    fn = os.path.join(data_dir,'generate_bg_png',f'{svg_id}.png')
    bg_img = PIL.Image.open(fn).convert("RGB") # background image
    H, W = bg_img.size[1], bg_img.size[0]
    #display(bg_img)
    visdata = {
        "renderer": renderer,
        "element_data": element_data,
        "bg_img": bg_img,
    }
    def show_sampling(predictions):
        fig=plt.figure(figsize=(50, 50))
        for iter in range(3):
            visdata["prediction"]=predictions[f"{svg_id}_{iter}"]
            img = visualize_prediction(**visdata).astype(np.uint8)
            plt.subplot(1, 3, iter+1)
            plt.imshow(img)
            plt.axis("off")
        plt.show()
    show_sampling(predictions)
    img = visualize_data(
        renderer,
        element_data,
        bg_img,
    )
    #display(PIL.Image.fromarray(img.astype(np.uint8)))
